In [7]:
import pandas as pd
import os
import numpy as np

In [10]:
def data_preprocessing(df):
    '''preprocess the price data into log return'''
    return (np.log(df) - np.log(df.shift(1))).dropna()

In [11]:
close_prices = data_preprocessing(pd.read_csv(os.path.join('data', 'Close.csv'), index_col=0, parse_dates=True))
gain = close_prices.values[1:] / close_prices.values[:-1]

g:\Code\Python\GitHub\Portfolio-Optimizer\.venv\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [16]:
close_prices.shape

(1042, 8)

In [18]:
np.hstack((np.ones((close_prices.shape[0]-1, 1)), close_prices.values[1:] / close_prices.values[:-1]))

g:\Code\Python\GitHub\Portfolio-Optimizer\.venv\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[  1.        ,   0.14726598,  -0.97044196, ...,   3.50025369,
         -1.99893816, -19.03932488],
       [  1.        ,  -1.17268178,   1.14595733, ...,  -3.43077212,
         -2.07279077,  -2.95131025],
       [  1.        ,   8.58206227,  -0.13973901, ...,   2.842935  ,
          2.09430525,   1.29091786],
       ...,
       [  1.        ,   0.35994901,   0.70225201, ...,  -0.28019302,
         -1.60579586,   0.03215113],
       [  1.        ,  -0.61156898,  -0.49518133, ...,  -2.78724507,
         -2.3416233 ,  20.03807267],
       [  1.        ,  -1.40679374,  -0.39700564, ...,   1.979147  ,
          0.97042576,   2.0929765 ]])

In [17]:
close_prices.values[1:].shape

(1041, 8)

In [14]:
close_prices.values[:-1]

array([[-7.03362310e-02, -5.49067987e-02, -2.15468000e-03, ...,
         1.50860882e-04, -5.90516084e-04, -6.48922625e-05],
       [-1.03581339e-02,  5.32838616e-02,  3.31443487e-03, ...,
         5.28051357e-04,  1.18040514e-03,  1.23550487e-03],
       [ 1.21467949e-02,  6.10610317e-02, -2.49974366e-03, ...,
        -1.81162387e-03, -2.44673287e-03, -3.64635818e-03],
       ...,
       [-6.36409367e-02, -6.24021930e-02,  1.22336096e-03, ...,
        -2.06006433e-03, -7.52237061e-04, -4.64556580e-03],
       [-2.29074924e-02, -4.38220654e-02, -8.73404116e-04, ...,
         5.77215638e-04,  1.20793916e-03, -1.49360203e-04],
       [ 1.40095117e-02,  2.16998684e-02,  7.41584701e-04, ...,
        -1.60884144e-03, -2.82853848e-03, -2.99289061e-03]])